# Convert Raster Data to DGGS Zones

In [1]:
import os.path

import folium
import json
import pystac_client
import rasterio
import shapely

from vgrid.conversion.raster2dggs import raster2a5, raster2rhealpix, raster2h3
from vgrid.utils.io import convert_to_output_format, download_file

## Identify the Study Area of Interest

In [2]:
with open("../manitoba_study_area/inputs/manitoba_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [3]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
)
result.matched()

73

In [4]:
item = next(result.items())
item

<Item id=9b27638f-6956-48e1-b66e-8f82df5cbbf5>

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57


In [5]:
for dggrs, dggrs_builder, dggrs_resolutions in [
    ("H3", raster2h3, [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    ("A5", raster2a5, [9]),
    ("rHEALPix", raster2rhealpix, [8, 9]),
]:
    for resolution in dggrs_resolutions:
        for item in result.items():
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            for band in ["rl", "rr"]:
                out_dir = os.path.join("./outputs", "manitoba_rcm_ard", dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                src_dir = os.path.join("./outputs", f"manitoba_rcm_ard", "src", date, band)
                src_path = os.path.join(src_dir, src_name)
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd"  # GeoPandas Dataframe
                )
                # save results
                for fmt in ["geojson", "parquet"]:
                    out_name = os.path.splitext(src_name)[0]
                    out_path = os.path.join(out_dir, f"{out_name}.{fmt}")
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

Converting to H3-L8 [band=rl]. Saving to [./outputs/manitoba_rcm_ard/H3/L8/2025-08-20/rl]...


Converting raster to H3: 100%|██████████| 29586/29586 [00:22<00:00, 1317.39 cells/s]


Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-08-20/rl/RCM2_OK3556292_PK3758929_1_SC30MCPB_20250820_124729_RL.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-08-20/rl/RCM2_OK3556292_PK3758929_1_SC30MCPB_20250820_124729_RL.parquet



KeyboardInterrupt

